URL index: [Home](https://zzz.bwh.harvard.edu/luna-walkthrough/) | [Data](https://zzz.bwh.harvard.edu/luna-walkthrough/data/) | [S1. File QC](https://zzz.bwh.harvard.edu/luna-walkthrough/p1/) | [S2. Signal QC](https://zzz.bwh.harvard.edu/luna-walkthrough/p2) | [S3. Staging](https://zzz.bwh.harvard.edu/luna-walkthrough/p3) | [S4. Artifacts](https://zzz.bwh.harvard.edu/luna-walkthrough/p4) | [S5. Analysis](https://zzz.bwh.harvard.edu/luna-walkthrough/p5)

Notebook index: [Index](../00_index.ipynb) | [S1. File QC](../p1/00_index.ipynb) | [S2. Signal QC](../p2/00_index.ipynb) | [S3. Staging](../p3/00_index.ipynb) | [S4. Artifacts](../p4/00_index.ipynb) | [S5. Analysis](../p5/00_index.ipynb)

---

# 1.7. Reviewing and harmonizing annotation labels

Walktrhough URL = [https://zzz.bwh.harvard.edu/luna-walkthrough/p1/anns/](https://zzz.bwh.harvard.edu/luna-walkthrough/p1/anns/)

In [1]:
import lunapi as lp
proj = lp.proj()
proj.sample_list( 's1.lst' )

initiated lunapi v0.1.1 <lunapi.lunapi0.luna object at 0x16e5f15f0> 

read 20 individuals from s1.lst


## Tabulating annotations

We've attached the previously generated `s1.lst` sample list.  Here we tabulate information on annotations.

In [2]:
res = proj.silent_proc( 'ANNOTS' )

In [3]:
proj.strata()

,Command,Strata
0,ANNOTS,ANNOT
1,ANNOTS,ANNOT_INST
2,ANNOTS,ANNOT_INST_T1_T2


This gives a table of all annotations, their count and (total) duration.

In [4]:
tbl = proj.table( 'ANNOTS' , 'ANNOT' ) 
tbl

,ID,ANNOT,COUNT,DUR
0,F09,0,48,1440.0
1,F10,0,183,5490.0
2,F09,1,44,1320.0
3,F10,1,81,2430.0
4,F09,2,477,14310.0
...,...,...,...,...
101,M04,W,245,7350.0
102,M05,W,98,2940.0
103,M06,W,212,6360.0
104,M07,W,363,10890.0


Counting the number of people with at least one of that annotation class. 

In [8]:
tbl[ 'ANNOT' ].value_counts()

ANNOT
N2            16
N1            15
N3            15
R             13
W             13
Arousal       10
5              2
3              2
2              2
1              2
SR             2
SW             2
SlpStg1        2
SlpStg2        2
SlpStg3        2
SlpStgREM      2
SlpStgWake     2
0              2
Name: count, dtype: int64

Above we note some "missing" annotations, e.g. `S1`, etc.   As described in the walkthrough, this reflects Luna's default mapping of stage labels.  We can turn that off and re-run to get all "original" / "as is" labels.

In [5]:
proj.var( 'annot-remap' , 'F' )
res = proj.silent_proc( 'ANNOTS' )
tbl = proj.table( 'ANNOTS' , 'ANNOT' ) 
tbl[ 'ANNOT' ].value_counts()

ANNOT
Arousal       10
N2             9
W              8
N1             8
N3             8
R              8
S1             6
S2             6
S3             6
Wake           4
REM            4
0              2
SlpStg2        2
SlpStgWake     2
SlpStgREM      2
SlpStg3        2
SR             2
SlpStg1        2
SW             2
1              2
5              2
3              2
2              2
n1             1
n2             1
n3             1
r              1
w              1
Name: count, dtype: int64

i.e. we now see `S1`, `S2`, etc included in the output (and correspondingly fewer people with `N1`, `N2`, etc labels).

We've defined a mapping file for these new/nonstandard stage labels:

In [6]:
%%sh
cat ../work/data/aux/amaps

remap	N1|1|S1|SlpStg1
remap	N2|2|S2|SlpStg2
remap	N3|3|S3|SlpStg3
remap	R|REM|5|SR|SlpStgREM
remap	W|Wake|0|SW|SlpStgWake


As for the channel aliases previously, we can `@include` these terms, i.e. will set all these special variables.

In [7]:
proj.include( '../work/data/aux/amaps' )

  setting remap = N1|1|S1|SlpStg1
  setting remap = N2|2|S2|SlpStg2
  setting remap = N3|3|S3|SlpStg3
  setting remap = R|REM|5|SR|SlpStgREM
  setting remap = W|Wake|0|SW|SlpStgWake


5

We can check they are included in the current _aliases_ table.

In [8]:
proj.aliases()

,Type,Preferred,"Case-insensitive, sanitized alias"
1,ANNOT,W,0
2,ANNOT,N1,1
3,ANNOT,N2,2
4,ANNOT,N3,3
5,ANNOT,R,5
6,ANNOT,R,REM
7,ANNOT,N1,S1
8,ANNOT,N2,S2
9,ANNOT,N3,S3
10,ANNOT,N1,SLPSTG1


Note: the previously defined _channel aliases_ are __not__ listed here, as we're running this in a separate notebook, which has a distinct kernel, implying a totally different instantation of lunapi.     Unlike the command-line version, in which every single Luna run is an atomic unit, it takes a little more care to keep track of what does persist (e.g. variables set across different cells in the same notebook, based on the order in which they were executed...) versus not (e.g. between different notebooks).

In any case, now we've set these explicit annotation remappings, we'll turn `annot-remap` back on and re-run `ANNOTS`.

In [9]:
proj.var( 'annot-remap' , 'T' )
res = proj.silent_proc( 'ANNOTS' )
tbl = proj.table( 'ANNOTS' , 'ANNOT' ) 
tbl[ 'ANNOT' ].value_counts()

ANNOT
N2         20
N1         19
N3         19
R          19
W          19
Arousal    10
Name: count, dtype: int64

That is, above we now see only the five stage labels, harmonized and defined for all individuals as expected (note, one person only had N2 sleep), in addition to the `Arousal` annotation.

## Making reformatted annotation files

The `WRITE-ANNOTS` command will save all annotation files - now with remapped labels - in consistent .annot formats in `../work/harm1/`.

In [10]:
#luna s1.lst @work/data/aux/amaps -o out.db -s WRITE-ANNOTS file=work/harm1/^.annot
res = proj.silent_proc( 'WRITE-ANNOTS file=../work/harm1/^.annot' ) 

We can check the new .annot files are now present alongside the EDFs.

In [11]:
%%sh
ls ../work/harm1/

F01.annot
F01.edf
F02.annot
F02.edf
F03.annot
F03.edf
F04.annot
F04.edf
F05.annot
F05.edf
F06.annot
F06.edf
F07.annot
F07.edf
F08.annot
F08.edf
F09.annot
F09.edf
F10.annot
F10.edf
M01.annot
M01.edf
M02.annot
M02.edf
M03.annot
M03.edf
M04.annot
M04.edf
M05.annot
M05.edf
M06.annot
M06.edf
M07.annot
M07.edf
M08.annot
M08.edf
M09.annot
M09.edf
M10.annot
M10.edf


Finally, we can take a peek at one of these files - e.g. standard .annot format (with time encoded in elapsed seconds from the EDF start).

In [17]:
%%sh 
head ../work/harm1/F01.annot

class	instance	channel	start	stop	meta
W	W	.	0.000	30.000	.
W	W	.	30.000	60.000	.
W	W	.	60.000	90.000	.
W	W	.	90.000	120.000	.
W	W	.	120.000	150.000	.
W	W	.	150.000	180.000	.
W	W	.	180.000	210.000	.
W	W	.	210.000	240.000	.
W	W	.	240.000	270.000	.


---

We can now move to the [final step](07_gen.ipynb) of making a new sample-list/project.